# AIFFEL 대전 1기 Going Deeper CV Lecture6. OCR
## Date: APRIL 22, 2021

# 1. OCR = Text detection + Text recognition
- OCR은 크게 **문자의 영역을 검출하는 Text detection** 그리고 **검출된 영역의 문자를 인식하는 Text recognition**으로 구분할 수 있다.  

1) **Text detection**은 단순히 Objection detection 기법 뿐 아니라 Segmentation 기법도 동원되며, 문자가 가지는 독특한 특성까지 감안한 형태로 지속적으로 발전

2) **Text recognition**은 검출된 영역의 문자가 무엇인지를 인식해 내는 과정

# 2. OCR 역사
딥러닝이 대중화되기 시작한 2010년대 초반으로부터 80년 이상이나 거슬러 올라간다고 합니다. 1928년 오스트리아의 Gustav Tauschek가 포토 센서를 활용한 패턴매칭을 통해 제작한 "Reading Machine"을 제작  

1931년 이스라엘의 Emanuel Goldberg가 "Statistical machine"의 특허를 내었는데, 이들이 최초의 동작하는 OCR 장치로 여겨지고 있습니다. 상용화된 OCR은 1980년대부터 본격적으로 등장

# 3. Tesseract OCR
1985년 휴렛 패커드사에서 개발이 시작되어 2005년에는 오픈소스로 출시  

 2006년부터는 구글의 후원을 받고 있는 오픈소스 OCR 엔진  
 
![Tesseract](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GC-6-L-01.png)  

위 그림에서,  
3단계는 입력이미지 추출과 전처리, 4번째 단계는 OCR 처리, 마지막 5번째 단계는 OCR의 출력 텍스트의 후처리로 구성.  


![T](https://aiffelstaticprd.blob.core.windows.net/media/images/ARCHIT1.max-800x600.png)  
위 그림은 Tesseract OCR을 통한 4단계의 과정을 정리한 것.  

1. `Adaptive Thresholding` 단계에서 입력영상의 이진화를 통해 우리가 흑백으로 보듯이 변환  
2. `Connected Component Analysis` 단계에서 문자영역을 검출  
3. `Find Lines and Words`에서 라인 또는 워드 단위를 추출  
4. `Recognize Word` 단계에서 Word 단위 이미지를 Text로 변환하기 위해 문자를 하나씩 인식하고 다시 결합  

# 4. Text Detection
## 1) Regression
![TextBoxes](https://aiffelstaticprd.blob.core.windows.net/media/images/architecture_of_textboxes.max-800x600.png)  

- TextBoxes는 2017년 AAAI에 공개된 논문입니다. 이미지 내에서 문자를 찾아내기 위한 방법에는 다양한 기법들이 활용  
- 위 논문에서는 딥러닝 기반의 Detection을 이용하여 단어 단위로 인식  
- 네트워크의 기본 구조는 [SSD:Single Shot Multibox Detector](https://arxiv.org/pdf/1512.02325.pdf)  

1. 일반적으로 단어들은 가로로 길기 때문에 Aspect ratio가 크다.  
2. 기존의 SSD에서는 Regression을 위한 Convolution layer에서 3x3 크기를 가지는데, 여기서는 **긴 단어의 Feature를 활용하기 위해서 1x5로 convolution filter**를 사용  
3. Anchor box 또한  1,2,3,5,7로 큰 Aspect ratio로 만들고 이를 vertical offset를 적용해 세로 방향으로 촘촘한 단어의 배열에 대응하도록 했다.  
![Aspect ratio](https://aiffelstaticprd.blob.core.windows.net/media/original_images/textbox_anchor.png)  

## 2) Segmentation
- [PixelLink: Detecting Scene Text via Instance Segmentation](https://arxiv.org/pdf/1801.01315.pdf)  

![Segmentaion](https://aiffelstaticprd.blob.core.windows.net/media/images/segmentation_map.max-800x600.png)  
- Semantic segmentation: 이미지 내의 영역을 class로 분리해내는 방법  

배경과 글자를 분리해 생각해보자.  
하지만 문자들은 매우 촘촘히 배열되어 있어서 글자 영역으로 찾아낸 뒤 이를 분리해내는 작업 또는 연력하는 작업을 더해서 원하는 최소단위로 만들어줘야 함.  

![PixelLink](https://aiffelstaticprd.blob.core.windows.net/media/images/architecture_pixellink.max-800x600.png)  

- **PixelLink**는 Text 영역을 찾아내는 segmentation과 함께 어느 방향으로 연결되는지 같이 학습을 하여 Text 영역간의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용하고 있다.  
- PixelLink의 전체적인 구조는 U-Net과 유사합니다. 하지만 output으로 총 9가지의 정보를 얻습니다.   

1. 위 그림의 녹색 부분: input, output  
2. output 중 하나는 Text/non-text Prediction을 위한 class segmentation map으로 해당 영역이 Text인지 Non-text인지 예측값을 의미하는 2개의 커널을 가짐.  
3. 나머지 8가지는 글자의 Pixel을 중심으로 인접한 8개의 Pixel에 대한 연결여부를 의마하는 16개의 커널로 이루어진 Link Prediction map입니다.  
4. 위 그림에서 conv 1x1, 2(16) 형태의 레이어가 U-Net 구조로 연결되어 인접 pixwl간 연결 구조가 지속적으로 유지되도록 하는 모델 구조임을 알 수 있다.  
5. 이를 통해 인접한 pixel이 중심 pixel과 단어단위로 연결된 pixel인지 아니면 분리된 pixel인지 알 수 있으므로, 문자영역이 단어단위로 분리된 Instance segmenation이 가능해진다.  

## 3) 최근 방법들
### 3-1)CRAFT
![craft](https://aiffelstaticprd.blob.core.windows.net/media/images/craft_affinity_map.max-800x600.png)  

**CRAFT는 Character 단위로 문자의 위치를 찾아낸 뒤 이를 연결하는 방식을 Segmentation기반으로 구현한 방법**  

1. 문자의 영역을 경계로 명확히 구분하지 않고 가우신안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자의 영역을 학습.  
2. 문자 단위 라벨을 가진 데이터셋이 많지 않기 때문에, 단어 단위의 정보만 있는 데이터셋에 대해 단어의 영역에 Inference를 한 후 얻어진 문자 단위의 위치를 다시 학습에 활용하는 weakly supervised learning을 활용.  

### 3-2) Pyramid Mask Text Detector
![PMTD](https://aiffelstaticprd.blob.core.windows.net/media/images/PMTD.max-800x600.png)  

1. PMTD(Pyramid Mask Text Detector)는 Mask-RCNN의 구조를 활용하여 먼저 Text영역을 Region proposal network로 찾아냄.  
2. Box head에서 더 정확하게 regression 및 classification을 하고 Mask head에서 Instance의 Segmentation을 하는 과정을 거침.  



# 4. Text recognition

## 1) Unsegmented Data
- 분리에 드는 비용이 비싸거나 어려워 Segmentation이 되어있지 않은 데이터  

## 2) CNN과 RNN의 만남 CRNN
![rcnn](https://aiffelstaticprd.blob.core.windows.net/media/original_images/crnn.png)  

![rcnn2](https://aiffelstaticprd.blob.core.windows.net/media/original_images/crnn_structure.png)  

1. Unsegmented Data들이 가진 주요한 특징 중 하나는 segment되어 있지 않은 하위데이터들끼리 시퀀스(sequence)를 이루고 있다는 점  
2. CNN 기반의 VGG 또는 ResNet 등의 네트워크를 이용해 Feature를 추출  
3. 추출된 Feature를 Map-to-Sequence를 통해 Sequence형태의 feature로 변환한 후 다양한 길이의 Input을 처리할 수 있는 RNN에 넣는다.  
4. RNN이 Feature로부터 문자를 인식하기 위해서는 문장 영역처럼 넓은 정보가 필요하기 때문에 LSTM으로 구성.  
5. 뒤의 정보도 필요하기 때문에 Bidirectional로 구성해서 Bidirectional LSTM을 사용.  
6. Bidirectional LSTM으로 step마다 나오는 결과는 Transcription Layer에서 문자로 변환

## 3) CTC(Connectionist Temporal Classification)
1.  Unsegmented data와 같이 Input과 Output이 서로 다른 Length의 Sequence가질 때, 이를 Align 없이 활용하는 방법  
2. 예시: "HELLO"라는 이미지가 들어오면 이것의 Output이 "HHHHEEELLLOOO..."와 같이 24자의 sequence를 보게 됨.  
3. 중복된 글자 구분 방법: 앞서 봤던 Label Encode에서 이렇게 같은 문자를 구분하기 위한 Blank를 중복된 라벨 사이를 구분하기 위해 넣어 준다.  

아래 그림은 Blank token을 '-'로 대신하여 Output을 만드는 Decoder를 의미합니다. Decode 후에 중복을 제거하고, 인식할 문자가 아닌 값을 지워주면 "HELLO"라는 결과를 얻을 수 있습니다.  

![CTC](https://aiffelstaticprd.blob.core.windows.net/media/original_images/wbHRk.png)  

## 4) TPS
- [Robust Scene Text Recognition With Automatic Rectification](https://arxiv.org/pdf/1603.03915.pdf)  

![TPS](https://aiffelstaticprd.blob.core.windows.net/media/images/spn.max-800x600.png)  

- 휘어진 글자를 잘 인식하는 방법  
- 위 링크의 논문 [Robust Scene Text Recognition With Automatic Rectification]에서는 Thin Plate Spline Transformation을 적용하여 입력이미지를 단어영역에 맞게 변형시켜 인식이 잘 되도록 해줍니다.  

![TPS2](https://aiffelstaticprd.blob.core.windows.net/media/images/thinplates-dy.max-800x600.png)  

1. Thin plate spline은 control point를 정의하고 해당 point들이 특정 위치로 옮겨졌을 때, 축방향의 변화를 interpolation하여 모든 위치의 변화를 추정  
2. 이를 통해서 전체 이미지 pixel의 변화를 control point로 만들어낼 수 있다.  
3. 이 논문(Robust Scene Text Recognition With Automatic Rectification)에서는 Control point 20개를 미리 정의합니다. 그리고 Spatial Transformer Network를 통해서 Control point가 얼마나 움직여야 하는지 예측하는 네트워크를 아래 그림과 같이 Recognition model 앞단에 붙여 입력이미지를 정방향으로 맞춰줍니다. TPS 연산은 미분가능한 연산이기 때문에 이 모듈을 Recognition model 앞단에 붙여서 학습이 바로 가능합니다.